In [ ]:
"""
Running (and fine-tuning) BERT based models
"""

In [1]:
# Fine tuning should be done with GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

C:\Program Files\Python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Program Files\Python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Program Files\Python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Program Files\Python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Found GPU at: /device:GPU:0


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from scipy.stats import spearmanr

from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import os
import joblib

from loader import loader_train, loader_test
from models import BaseEstimator
from sklearn.pipeline import make_pipeline

from transforms import BERTProcessing
from models import BERTModel

from evaluation import spearman_measure

Using TensorFlow backend.


# Load data

In [ ]:
# Splitting the dataset (0.75/0.25) for the meta regression model that use BERT models predictions
dfX, y = loader_train()
dfX_train, dfX_val, y_train, y_val = train_test_split(dfX, y, shuffle=True, random_state=1)
dfX_test = loader_test()

In [ ]:
joblib.dump(dfX_train, 'dfX_train.dump')
joblib.dump(dfX_val, 'dfX_val.dump')
joblib.dump(y_train, 'y_train.dump')
joblib.dump(y_val, 'y_val.dump')

# Build and fit model

In [ ]:
model_name = 'xlnet-large-cased'
model_type = 'xlnet'
model = make_pipeline(BERTProcessing(model_name, model_type), BERTModel(model_name, model_type))

In [ ]:
model.fit(dfX_train,y_train)
y_pred = model.predict(dfX_val)

In [ ]:
joblib.dump(y_pred, 'pred_xlnet_large_val.dump')

In [ ]:
y_pred_test = model.predict(dfX_test)
joblib.dump(y_pred_test, 'pred_xlnet_large_test.dump')

In [4]:
!jupyter nbconvert --to html BERT_based_models.ipynb

[NbConvertApp] Converting notebook BERT_based_models.ipynb to html
[NbConvertApp] Writing 286904 bytes to BERT_based_models.html
